In [ ]:
# === Cell 1: Setup & Data Config ===
import os, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from google.colab import drive
drive.mount('/content/drive')
# Path to your dataset with two subfolders (e.g., negative/, positive/)
DATA_DIR = "/content/drive/MyDrive/Images/Images"

# Training configuration
IMG_SIZE = (320, 320)   # Try (320, 320). If you get OOM, fall back to (224, 224) or (288, 288)
BATCH    = 16
SEED     = 13
AUTOTUNE = tf.data.AUTOTUNE

# Quick sanity check
print("Exists?", os.path.isdir(DATA_DIR))
if os.path.isdir(DATA_DIR):
    print("First items:", os.listdir(DATA_DIR)[:10])
else:
    print("WARNING: DATA_DIR not found. Please update DATA_DIR.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Exists? True
First items: ['No_Appendicitis_Images', 'Appendicitis_Images']


In [ ]:
# === Cell 2: Build Datasets (train/val) ===
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR, labels="inferred", label_mode="binary",
    validation_split=0.20, subset="training", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR, labels="inferred", label_mode="binary",
    validation_split=0.20, subset="validation", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH
)

# If you have a separate test set, load it similarly. Otherwise we reuse val_ds below.
test_ds = val_ds

def prep(ds, training=False):
    # Shuffle only during training; cache+prefetch for performance
    if training:
        ds = ds.shuffle(1024, seed=SEED, reshuffle_each_iteration=True)
    return ds.cache().prefetch(AUTOTUNE)

train_ds = prep(train_ds, training=True)
val_ds   = prep(val_ds)
test_ds  = prep(test_ds)


Found 1721 files belonging to 2 classes.
Using 1377 files for training.
Found 1721 files belonging to 2 classes.
Using 344 files for validation.


In [ ]:
# === Cell 3: Compute Class Weights (handles imbalance) ===
neg, pos = 0, 0
for _, yb in train_ds.unbatch().take(1_000_000):  # large limit to cover full dataset
    if int(yb.numpy()[0]) == 1:
        pos += 1
    else:
        neg += 1
total = max(1, pos+neg)
cw = {
    0: total/(2*max(1,neg)),
    1: total/(2*max(1,pos)),
}
print("Class counts:", {"neg":neg, "pos":pos})
print("Using class weights:", cw)


Class counts: {'neg': 1074, 'pos': 303}
Using class weights: {0: 0.6410614525139665, 1: 2.272277227722772}


In [ ]:
# === Cell 4: Data Augmentations (light/medical-safe) ===
data_augment = keras.Sequential([
    layers.RandomFlip("horizontal"),   # remove if left/right matters clinically
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="augment")


In [ ]:
# === Cell 5: Build DenseNet121 (Stage 1: Frozen base) ===
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

base = DenseNet121(include_top=False, weights="imagenet", input_shape=IMG_SIZE+(3,))
base.trainable = False

inp = layers.Input(shape=IMG_SIZE+(3,))
x   = data_augment(inp)
x   = layers.Lambda(preprocess_input)(x)
x   = base(x, training=False)
x   = layers.GlobalAveragePooling2D()(x)
x   = layers.Dropout(0.3)(x)  # increase to 0.4-0.5 if overfitting
out = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inp, out)

steps_per_epoch = int(np.ceil(train_ds.cardinality().numpy()))
total_steps     = steps_per_epoch * 5   # ~5 epochs in stage 1

schedule = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=3e-4, decay_steps=total_steps
)
opt = keras.optimizers.AdamW(learning_rate=schedule, weight_decay=1e-4)

model.compile(
    optimizer=opt,
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.05),
    metrics=[
        keras.metrics.BinaryAccuracy(name="acc"),
        keras.metrics.AUC(name="auc"),
        keras.metrics.AUC(name="auprc", curve="PR"),
        keras.metrics.Precision(name="prec"),
        keras.metrics.Recall(name="rec"),
    ],
)

cbs = [
    keras.callbacks.ModelCheckpoint("densenet_stage1.keras", save_best_only=True, monitor="val_auc", mode="max"),
    keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=3, restore_best_weights=True),
]

model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 320, 320, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ augment (Sequential)            │ (None, 320, 320, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ (None, 320, 320, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 10, 10, 1024)   │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,038,529 (26.85 MB)

 Trainable params: 1,025 (4.00 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [ ]:
# === Cell 6: Train Stage 1 (Frozen) ===
history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,                # can increase slightly if still improving
    class_weight=cw,
    callbacks=cbs
)


Epoch 1/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 632s 7s/step - acc: 0.5779 - auc: 0.4977 - auprc: 0.2325 - loss: 0.8529 - prec: 0.2225 - rec: 0.3274 - val_acc: 0.5494 - val_auc: 0.5582 - val_auprc: 0.2500 - val_loss: 0.6921 - val_prec: 0.2683 - val_rec: 0.5570
Epoch 2/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 615s 7s/step - acc: 0.5052 - auc: 0.5086 - auprc: 0.2546 - loss: 0.7708 - prec: 0.2248 - rec: 0.4762 - val_acc: 0.5727 - val_auc: 0.5948 - val_auprc: 0.2771 - val_loss: 0.6894 - val_prec: 0.2927 - val_rec: 0.6076
Epoch 3/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 619s 7s/step - acc: 0.5302 - auc: 0.5398 - auprc: 0.2624 - loss: 0.7513 - prec: 0.2609 - rec: 0.5355 - val_acc: 0.5727 - val_auc: 0.6123 - val_auprc: 0.2897 - val_loss: 0.6877 - val_prec: 0.3068 - val_rec: 0.6835
Epoch 4/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 645s 7s/step - acc: 0.5378 - auc: 0.5310 - auprc: 0.2714 - loss: 0.7617 - prec: 0.2565 - rec: 0.5192 - val_acc: 0.4448 - val_auc: 0.6164 - val_auprc: 0.2931 - val_loss: 0.7214 - val_prec: 0.2667 - val_rec: 0.8101


In [ ]:
# === Cell 7: Fine-tune Stage 2 (unfreeze deeper layers) ===
# 1) Unfreeze the last ~160 layers (adjust count if needed)
base.trainable = True
n_layers = len(base.layers)
unfreeze_last = 160  # tweak as you like
freeze_until = max(0, n_layers - unfreeze_last)

for i, layer in enumerate(base.layers):
    layer.trainable = (i >= freeze_until)
    # (Optional but common) keep BatchNorm layers frozen during fine-tuning
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

# 2) Optimizer: plain float LR so callbacks can adjust it
# Tip: fine-tuning often benefits from a smaller LR like 3e-5 to 1e-5
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)

model.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(curve='PR', name='auprc'),
        tf.keras.metrics.Precision(name='prec'),
        tf.keras.metrics.Recall(name='rec'),
    ],
)

# 3) Callbacks (consistent monitors)
cbs_ft = [
    keras.callbacks.ModelCheckpoint(
        "densenet_finetuned.keras", save_best_only=True,
        monitor="val_auc", mode="max"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min", factor=0.5,
        patience=2, min_lr=1e-6, verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_auc", mode="max",
        patience=5, restore_best_weights=True
    ),
]

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=cbs_ft,
    verbose=1,
)


Epoch 1/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 747s 8s/step - accuracy: 0.7569 - auc: 0.5832 - auprc: 0.3266 - loss: 0.5584 - prec: 0.4020 - rec: 0.1179 - val_accuracy: 0.7703 - val_auc: 0.7016 - val_auprc: 0.4114 - val_loss: 0.5058 - val_prec: 0.0000e+00 - val_rec: 0.0000e+00 - learning_rate: 3.0000e-05
Epoch 2/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 736s 8s/step - accuracy: 0.7776 - auc: 0.7005 - auprc: 0.4482 - loss: 0.4951 - prec: 0.6588 - rec: 0.0865 - val_accuracy: 0.7703 - val_auc: 0.7423 - val_auprc: 0.4777 - val_loss: 0.5014 - val_prec: 0.0000e+00 - val_rec: 0.0000e+00 - learning_rate: 3.0000e-05
Epoch 3/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 738s 8s/step - accuracy: 0.7866 - auc: 0.7451 - auprc: 0.4906 - loss: 0.4730 - prec: 0.6634 - rec: 0.1582 - val_accuracy: 0.7733 - val_auc: 0.7399 - val_auprc: 0.4686 - val_loss: 0.5042 - val_prec: 1.0000 - val_rec: 0.0127 - learning_rate: 3.0000e-05
Epoch 4/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 713s 8s/step - accuracy: 0.7876 - auc: 0.7779 - auprc: 0.5269 - loss: 0.4522 - 

In [ ]:
# === Cell 8: Threshold Search (Youden J) on Validation ===
import numpy as np
y_true, y_score = [], []
for Xb, yb in val_ds:
    y_true.append(yb.numpy().ravel())
    y_score.append(model.predict(Xb, verbose=0).ravel())
y_true  = np.concatenate(y_true)
y_score = np.concatenate(y_score)

from sklearn.metrics import roc_curve, auc
fpr, tpr, thr = roc_curve(y_true, y_score)
best_idx = np.argmax(tpr - fpr)  # Youden J statistic
best_thr = float(thr[best_idx])

print(f"Best threshold (Youden J): {best_thr:.3f}, Val AUC={auc(fpr,tpr):.3f}")


Best threshold (Youden J): 0.085, Val AUC=0.791


In [ ]:
# === Cell 9: Optional Test-Time Augmentation (TTA) and Evaluation ===
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def tta_predict(ds, n=5):
    preds = []
    for _ in range(n):
        batch_preds = []
        for Xb, _ in ds:
            batch_preds.append(model.predict(Xb, verbose=0))
        preds.append(np.concatenate(batch_preds).ravel())
    return np.mean(np.stack(preds, axis=0), axis=0)

use_tta = True
if use_tta:
    y_t = []
    for _, yb in test_ds:
        y_t.append(yb.numpy().ravel())
    y_t = np.concatenate(y_t)
    y_p = tta_predict(test_ds, n=5)
else:
    # if you want to evaluate on val with no TTA, reuse the arrays from Cell 8
    y_t, y_p = y_true, y_score

y_pred = (y_p >= best_thr).astype(int)

cm = confusion_matrix(y_t, y_pred)
print("Confusion matrix:\n", cm)
print(classification_report(y_t, y_pred, digits=3))
print("Accuracy @best_thr:", accuracy_score(y_t, y_pred))


Confusion matrix:
 [[183  82]
 [ 16  63]]
              precision    recall  f1-score   support

         0.0      0.920     0.691     0.789       265
         1.0      0.434     0.797     0.562        79

    accuracy                          0.715       344
   macro avg      0.677     0.744     0.676       344
weighted avg      0.808     0.715     0.737       344

Accuracy @best_thr: 0.7151162790697675
